In [14]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
import os

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile
from pydub import AudioSegment

In [10]:
# Load the model.
model = hub.load('https://tfhub.dev/google/yamnet/1')

In [11]:
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
  """Returns list of class names corresponding to score vector."""
  class_names = []
  with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      class_names.append(row['display_name'])

  return class_names

class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(class_map_path)


In [7]:
#wav_file_name = 'speech_whistling2.wav'
wav_file_name = '/home/mdv/programming/personal/Pickhacks2024/Screaming girl on slingshot-[AudioTrimmer.com].wav'
# wav_file_name = 'miaow_16k.wav'
sample_rate, wav_data = wavfile.read(wav_file_name, 'rb')

waveform = wav_data / tf.int16.max

In [12]:
# def upload_audio():
#     audio_file = request.files['file']
#     # Save the uploaded file
#     file_path = 'uploads/' + audio_file.filename
#     audio_file.save(file_path)

#     # Convert to mono and resample to 16kHz if necessary
#     converted_file_path = convert_audio(file_path)
    
#     return ({"file_path": converted_file_path})

def convert_audio(file_path):
    sound = AudioSegment.from_file(file_path)
    # Ensure mono channel
    sound = sound.set_channels(1)
    # Ensure 16kHz sample rate
    sound = sound.set_frame_rate(16000)
    # Save the converted audio
    converted_file_path = 'uploads/converted_' + os.path.basename(file_path)
    sound.export(converted_file_path, format="wav")
    return converted_file_path

In [16]:
wav_file_name = '/home/mdv/programming/personal/Pickhacks2024/Screaming girl on slingshot-[AudioTrimmer.com].wav'
conv_file_path = convert_audio(wav_file_name)

In [ ]:
sample_rate, wav_data = wavfile.read(conv_file_path, 'rb')

waveform = wav_data / tf.int16.max

# Run the model, check the output.
scores, embeddings, spectrogram = model(waveform)
scores_np = scores.numpy()
spectrogram_np = spectrogram.numpy()
infered_class = class_names[scores_np.mean(axis=0).argmax()]
# print(f'The main sound is: {infered_class}')
print(infered_class)


In [36]:
# Run the model, check the output.
scores, embeddings, spectrogram = model(waveform)

# Convert scores to numpy array
scores_np = scores.numpy()

# Get the indices of top 10 labels with highest scores
top_10_indices = scores_np.mean(axis=0).argsort()[-10:][::-1]

# Print the top 10 labels and their probabilities
print("Top 10 labels:")
for idx in top_10_indices:
    label = class_names[idx]
    probability = scores_np.mean(axis=0)[idx]
    print(f"{label}: {probability}")


Top 10 labels:
Water: 0.1983717828989029
Ocean: 0.1787852644920349
Screaming: 0.1637938916683197
Waves, surf: 0.1486632227897644
Splash, splatter: 0.10543322563171387
Slosh: 0.09673410654067993
Speech: 0.0707528218626976
Liquid: 0.06571026146411896
Whistle: 0.05388862267136574
Whistling: 0.053471896797418594
